# 保持会话（Session）

本笔记系统介绍在 Python 爬虫中使用 `requests.Session` 保持会话与提升性能的常见实践：
- 自动管理 Cookie
- 统一 Header/超时/重试
- 连接池复用
- 登录状态保持
- 并发注意事项与调试技巧

## 目录
1. 为什么需要 Session
2. Session 与直接 requests.get 的差异
3. 基本用法：自动管理 Cookie 与 Header 继承
4. 常见场景：登录后访问 + CSRF Token
5. 统一配置：连接池、重试、超时、默认 headers
6. Cookie 持久化与加载（cookiejar）
7. 统一 Proxies / Auth / Headers 设置
8. Hooks：响应后处理与调试
9. 并发使用注意事项（线程 / 进程 / 协程）
10. 关闭与资源释放
11. 常见坑与调试技巧
12. 小结与参考链接

## 1. 为什么需要 Session
- 维持登录状态：自动保存服务器通过 Set-Cookie 设置的 Cookie。
- 复用 TCP 连接：减少三次握手与 TLS 握手开销，提高吞吐与延迟表现。
- 集中配置：统一 headers / proxies / auth / retries / timeout，减少重复代码。
- 可扩展：通过挂载自定义 Adapter 控制连接池大小、重试策略等。
- 降低复杂度：不用手动在每次请求中传入 Cookie 与公共 Header。

## 2. Session 与直接 requests.get 的差异
| 对比点 | 直接 requests.get | Session.get |
|--------|------------------|------------|
| Cookie 自动持久 | 否（每次独立） | 是（同一个 Session 周期内共享） |
| 连接复用 | 依赖 urllib3 的短暂复用，但生命周期短 | 明确复用，连接池配置可调 |
| 统一默认 headers/proxies | 每次手动传 | 设置一次，后续继承 |
| 自定义重试策略 | 需手动 adapter | 容易 mount 到 Session |
| 关闭资源 | 无显式关闭 | 可显式关闭释放连接 |

注意：`Session` 并不是跨进程/跨机器的“长登录”，只是当前 Python 进程里的状态容器。

In [1]:
# 对比：普通 requests 与 Session 在 Cookie 自动保存上的差异
import requests
from requests import Session

# 普通方式：第二次请求不会自动带上第一次的 cookie
r1 = requests.get('https://httpbin.org/cookies/set?demo=1', timeout=10)
r2 = requests.get('https://httpbin.org/cookies', timeout=10)
print('普通方式 -> 第二次返回 cookies:', r2.json())

# Session 方式：第二次自动带上之前保存的 cookie
s = Session()
s.get('https://httpbin.org/cookies/set?demo=1', timeout=10)
r_session = s.get('https://httpbin.org/cookies', timeout=10)
print('Session 方式 -> 第二次返回 cookies:', r_session.json())

普通方式 -> 第二次返回 cookies: {'cookies': {}}
Session 方式 -> 第二次返回 cookies: {'cookies': {'demo': '1'}}


## 3. 基本用法：自动管理 Cookie 与 Header 继承
- 在 Session 上设置的 headers 会作为默认值，被后续请求继承，可以在单次请求中覆盖。
- Session.cookies 是一个 RequestsCookieJar，可直接 `get_dict()` 查看。
- 超时（timeout）最好每次请求都传；若要统一，可以封装一个 helper。

In [3]:
from requests import Session

s = Session()
s.headers.update({
    'User-Agent': 'Mozilla/5.0 (SessionDemo)',
    'Accept': 'application/json'
})
# 设置一个初始 Cookie（也可以通过访问接口让服务器设置）
s.cookies.set('local_pref', 'zh')

r = s.get('https://httpbin.org/headers', timeout=10)
print('返回的 UA:', r.json()['headers'].get('User-Agent'))
print('当前 session cookies:', s.cookies.get_dict())
# 单次请求覆盖 UA
r2 = s.get('https://httpbin.org/headers', headers={'User-Agent': 'OverrideUA/1.0'}, timeout=10)
print('覆盖后 UA:', r2.json()['headers'].get('User-Agent'))

返回的 UA: Mozilla/5.0 (SessionDemo)
当前 session cookies: {'local_pref': 'zh'}
覆盖后 UA: OverrideUA/1.0


## 4. 常见场景：登录后访问 + CSRF Token
模拟：先获取一个伪造的 token（或从响应中解析），再在后续 POST 中使用。真实站点中往往需要解析隐藏字段、meta 标签或响应 JSON。

In [5]:
from requests import Session

s = Session()
# 假设我们先访问一个页面获得 token，这里用 httpbin 的 /uuid 模拟返回 JSON 中的 uuid 当 token
token_resp = s.get('https://httpbin.org/uuid', timeout=10).json()
csrf_token = token_resp.get('uuid')
print('模拟获取到的 CSRF Token:', csrf_token)

# 后续请求携带 token（真实场景可能放在 header 或表单中）
payload = {'user': 'demo', 'action': 'save', 'csrf': csrf_token}
post_resp = s.post('https://httpbin.org/post', json=payload, timeout=10)
print('POST 回显的 json form/json 字段:', post_resp.json().get('json'))

模拟获取到的 CSRF Token: 0492175d-c0f7-49ca-8b1e-504fae57ddda
POST 回显的 json form/json 字段: {'action': 'save', 'csrf': '0492175d-c0f7-49ca-8b1e-504fae57ddda', 'user': 'demo'}


## 5. 统一配置：连接池、重试、超时、默认 headers
通过挂载自定义 `HTTPAdapter` 可以：
- 设置连接池大小（pool_connections / pool_maxsize）
- 设置是否在池满时阻塞（pool_block）
- 配置 urllib3 的 Retry 策略（status_forcelist、backoff 等）

建议把这些封装为一个创建 Session 的工厂函数，便于复用。

In [6]:
from requests import Session
from requests.adapters import HTTPAdapter
from urllib3.util import Retry

def create_session(
    retries_total=5,
    backoff_factor=0.3,
    status_forcelist=(429, 500, 502, 503, 504),
    allowed_methods=("HEAD", "GET", "OPTIONS", "POST"),
    pool_connections=10,
    pool_maxsize=20,
    pool_block=True,
    default_headers=None,
):
    s = Session()
    if default_headers:
        s.headers.update(default_headers)
    retry_cfg = Retry(
        total=retries_total,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
        allowed_methods=allowed_methods,
        raise_on_status=False,
    )
    adapter = HTTPAdapter(
        max_retries=retry_cfg,
        pool_connections=pool_connections,
        pool_maxsize=pool_maxsize,
        pool_block=pool_block,
    )
    s.mount('https://', adapter)
    s.mount('http://', adapter)
    return s

sess = create_session(default_headers={'User-Agent': 'SessionFactory/1.0'})
resp = sess.get('https://httpbin.org/get', timeout=10)
print('自定义工厂创建的 session UA:', resp.json()['headers'].get('User-Agent'))

自定义工厂创建的 session UA: SessionFactory/1.0


## 6. Cookie 持久化与加载（cookiejar）
可与标准库 `http.cookiejar` 的 `MozillaCookieJar` / `LWPCookieJar` 配合，把登录后的 Cookie 保存到文件，下次启动可恢复（需注意过期）。
与已有笔记《设置 Cookie》内容互补：此处演示绑定到 Session 并重新加载。

In [7]:
import os
from http.cookiejar import MozillaCookieJar
from requests import Session

cookie_file = 'cookies.txt'  # 与现有文件共用
jar = MozillaCookieJar(cookie_file)
s = Session()
s.cookies = jar  # 绑定
# 访问设置 Cookie
s.get('https://httpbin.org/cookies/set?persist_session=1', timeout=10)
jar.save(ignore_discard=True, ignore_expires=True)
print('已保存 Cookie 到:', os.path.abspath(cookie_file))

# 新的 Session 恢复 Cookie
s2 = Session()
jar2 = MozillaCookieJar(cookie_file)
jar2.load(ignore_discard=True, ignore_expires=True)
s2.cookies = jar2
print('恢复后的 cookies 字典:', s2.cookies._cookies.get('httpbin.org'))  # 仅演示结构

ReadTimeout: HTTPSConnectionPool(host='httpbin.org', port=443): Read timed out. (read timeout=10)

## 7. 统一 Proxies / Auth / Headers 设置
- 可在 Session 上直接设置 `s.proxies`、`s.auth`、`s.headers`。
- 适合需要在大量请求中复用相同代理或认证的场景。
- 注意：敏感凭据不要硬编码到源码，使用环境变量或配置文件。

In [ ]:
from requests import Session
from requests.auth import HTTPBasicAuth

s = Session()
s.headers.update({'User-Agent': 'BulkRequests/1.0'})
s.proxies.update({  # 示例代理（需替换为真实可用）
    'http': 'http://127.0.0.1:10809',
    'https': 'http://127.0.0.1:10809',
})
s.auth = HTTPBasicAuth('user', 'pass')  # 某些站点需要基本认证
try:
    r = s.get('https://httpbin.org/get', timeout=10)
    print('带统一配置的返回 origin:', r.json().get('origin'))
except Exception as e:
    print('统一配置请求失败(可能代理不可用):', e)

## 8. Hooks：响应后处理与调试
`Session` 支持在请求中传 `hooks={'response': [func, ...]}` 进行回调。常用于：
- 统一日志与指标采集
- 自动检测异常模式（如未登录重定向）
- 提取特定 header/token 进行链式操作

In [8]:
from requests import Session

def log_hook(r, *args, **kwargs):
    print(f'[HOOK] {r.request.method} {r.request.url} -> {r.status_code}')
    return r  # 必须返回 r 或不返回（requests 会继续处理）

s = Session()
r = s.get('https://httpbin.org/get', timeout=10, hooks={'response': [log_hook]})
print('响应 JSON 中的 url 字段:', r.json().get('url'))

[HOOK] GET https://httpbin.org/get -> 200
响应 JSON 中的 url 字段: https://httpbin.org/get


## 9. 并发使用注意事项（线程 / 进程 / 协程）
- 线程：`requests.Session` 对“并发读”通常安全，但对 headers/cookies 的“并发写”非线程安全，避免多个线程同时修改。
- 进程：进程间不能共享同一个 Session 对象（对象不会自动跨进程同步），需在子进程各自创建。
- 协程：`requests` 是阻塞的，在 asyncio 下建议用 `httpx` 或 `aiohttp`。
- 高并发场景：可为每个线程创建独立 Session，或使用连接池增大 `pool_maxsize`。

下面示例展示多线程共享只读 Session 做 GET（不修改其状态），并统计耗时。

In [ ]:
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from requests import Session

s = Session()
urls = [f'https://httpbin.org/get?i={i}' for i in range(5)]
start = time.time()
def fetch(u):
    r = s.get(u, timeout=10)
    return r.status_code

with ThreadPoolExecutor(max_workers=5) as ex:
    futures = [ex.submit(fetch, u) for u in urls]
    for fu in as_completed(futures):
        print('并发请求状态码:', fu.result())
print('总耗时:', round(time.time() - start, 2), 's')
# 注意：如果在 fetch 中修改 s.headers 或 s.cookies 就可能产生竞态风险。

## 10. 关闭与资源释放
- 使用完 Session 后调用 `s.close()` 主动释放连接，尤其在创建大量短生命周期 Session 时。
- 可用上下文管理器：`with Session() as s: ...` 自动在退出时关闭。
- 长时间运行的爬虫：定期评估是否需要刷新 Session（例如防止因大量遗留连接导致资源占用）。

In [ ]:
from requests import Session

with Session() as s:
    r = s.get('https://httpbin.org/get', timeout=10)
    print('上下文管理器返回状态码:', r.status_code)
# 退出 with 块后 Session 已自动关闭。

## 11. 常见坑与调试技巧
1. Cookie 不生效：检查 domain/path/secure/HttpOnly；确认后续请求 URL 与 Cookie 作用域匹配。
2. 重试不触发：确认使用了允许的方法（allowed_methods）且状态码在 status_forcelist。
3. Session 被滥用：不要把同一个 Session 用在完全不同的业务上下文（避免 Cookie/headers 污染）。
4. 代理 + 重试：某些代理错误（ProxyError）可能不会按 status 触发重试，需在 Retry 的 `retry_on_exception`（urllib3 2.x 可通过参数）或自行装饰。
5. 超时未设置：默认无 timeout 可能导致爬虫挂住，务必在每次请求或封装层统一传入 timeout。
6. 线程安全：并发修改 Session 状态（headers/cookies）可能导致数据竞争，必要时为每个线程独立创建 Session。
7. SSL 验证：企业内网或自签证书需传入 `verify=False` 或指定 CA 证书文件，关闭验证需谨慎。
8. 重定向循环：使用 r.history 调试；必要时 `allow_redirects=False`。
9. 性能诊断：启用 logging（`import logging; logging.basicConfig(level=logging.DEBUG)`）查看连接与重试行为。
10. 不要在 Session 初始化后再频繁 mount 适配器（可能已建立的连接不会受影响），提前封装好工厂。

## 12. 小结与参考链接
- `requests.Session` 提供了保持会话状态、提升性能与统一配置的基础能力。
- 正确使用：统一封装创建、显式 timeout、避免跨业务污染、关注并发安全。
- 进阶：结合 Retry、cookiejar 持久化、hooks、连接池参数、日志。

参考：
- requests 官方文档：https://requests.readthedocs.io/en/latest/
- urllib3 Retry：https://urllib3.readthedocs.io/en/stable/reference/urllib3.util.html#module-urllib3.util.retry
- http.cookiejar 标准库：https://docs.python.org/3/library/http.cookiejar.html